<a href="https://colab.research.google.com/github/hamagami/anomaly-detection/blob/main/02_01_%E3%82%AB%E3%82%A4%E4%BA%8C%E4%B9%97%E6%A4%9C%E5%AE%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# カイ二乗検定による異常検知
得られるデータがある分布をなしているとわかっている場合は，注目するデータが全体の中でどれくらい異質な存在かを厳密に評価することができます．特に，独立変数が正規分布の場合の異常度はカイ二乗分布に従うことがわかっています．この性質を利用して，得られたサンプルの異常度を厳密に評価できます．


In [ ]:
import matplotlib.pyplot as plt 
import numpy as np                
from scipy.stats import norm  #正規分布
from scipy.stats import chi2  #カイ二乗分布

パラメータから正規分布データを作成しておく

In [ ]:
mean=3.15 #平均値
var=2.114 #分散
dnum=1000 #データ数
data=np.random.normal(mean, var, dnum)

In [ ]:
param = norm.fit(data) #dataに対して正規分布のフィッティングを試してみる
x = np.linspace(-5,12,100) 
pdf_fitted = norm.pdf(x, loc=param[0], scale=param[1]) # 

In [ ]:
plt.hist(data,bins=100,density=1)
plt.plot(x,pdf_fitted)
plt.show()  #生成したdataがちゃんと正規分布に沿っていることを確認

作成したデータのそれぞれに対してカイ二乗分布による異常度の評価を行う

In [ ]:
# 様々なカイ二乗分布の確認
x = np.linspace(0,20,100) # x軸の値を作成
for f in range(1,5):
  chi_2 = chi2.pdf(x, f, scale=1) # 自由度f, スケール因子1のχ2乗分布　≒ N>>1のときの自由度(f, N-1)のF分布
  plt.plot(x,chi_2,label="df="+str(f)) # χ2乗分布の表示
plt.legend()
plt.show()
for f in range(6,10):
  chi_2 = chi2.pdf(x, f, scale=1) # 自由度f, スケール因子1のχ2乗分布　≒ N>>1のときの自由度(f, N-1)のF分布
  plt.plot(x,chi_2,label="df="+str(f)) # χ2乗分布の表示
plt.legend()
plt.show()
# 自由度kが増えると次第に正規分布に近づく

In [ ]:
th_prob=0.95 #95%までを正常とする
th  = chi2.ppf(th_prob, 1, scale=1) #閾値を求める。自由度１の場合は3.841

x = np.linspace(0,5,100) # x軸の値を作成
chi_2 = chi2.pdf(x, 1, scale=1) # 自由度1, スケール因子1のχ2乗分布　≒ N>>1のときの自由度(1, N-1)のF分布
xano=x[x> th]
ano=chi_2[x> th]

plt.plot(x,chi_2) # χ2乗分布の表示
plt.fill_between(xano,ano,color="red")
plt.title("Chisquare")
plt.xlabel("Abnomarity")
plt.ylabel("Probability Density Function (PDF)")
plt.show()

In [ ]:
#dataと　閾値:th
def predict(X, param, th):
    a = ((X - param[0])/param[1])**2 # あらかじめ求めておいた平均と分散から標準化をしておく．自由度1のカイ2乗分布は正規分布の2乗とほぼ等価
    pred = np.zeros_like(a) # 予測したラベル（すべて0（悪性）として初期化）
    pred[a <= th] = 1 # 異常値が閾値を下回る場合は正常
    return pred # 予測を返す

In [ ]:

p= predict(data,param,th) #判定結果
ano=data[p==0] #結果が0(異常）だけを抽出

In [ ]:
plt.plot(data,linestyle='None',marker=".")
plt.plot(ano,linestyle='None',marker=".")
plt.show()
plt.hist(data,bins=100,label="Normal("+str(th_prob)+")")
plt.hist(ano,bins=100,label="Anomaly("+str(1.0-th_prob)+")")
plt.legend()

plt.show()